In [3]:
import os
import csv
import ast
import json
import statistics

import numpy as np
import pandas as pd
# import seaborn as sns
# import matplotlib.pyplot as plt

from datetime import datetime, timedelta

In [8]:
database = "/Volumes/mollyT7/MOXA/"
# database = "/home/wmnlab/Documents/r12921105"
date = "20240411"
date_with_dash = "2024-04-11"
port = "5200"
phone_time = "0629"
time = "1428"
device = "sm00"
if int(port) % 2 == 0:
    sent_file_name = f"log_{date}_{phone_time}_{port}_client"
    received_file_name = f"log_{date}_{time}_{port}_server"
else:
    sent_file_name = f"log_{date}_{time}_{port}_server"
    received_file_name = f"log_{date}_{phone_time}_{port}_client"
path = f"{database}/{date_with_dash}/QUIC-inf/{device}/#01/"

In [9]:
sync_file_name = f"{database}/{date_with_dash}/time_sync_{device}.json"
# sync_file = path + "raw/" + sync_file_name
with open(sync_file_name, 'r') as file:
    data = json.load(file)

# Extract values from the dictionary
values = list(data.values())
mean_diff = values[0] * 1000

### Transform to JSON & CSV file
Process the qlog file to json file & csv file.

In [10]:
def QlogToJsonEntry(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    # Add commas between lines
    json_str = ",".join(lines)
    # Surround the entire string with square brackets to create a JSON array
    json_str = "[" + json_str + "]"
    # Load the JSON array
    json_entry = json.loads(json_str)
    
    return json_entry

def QlogToJson(json_entry, json_file_path):
    with open(json_file_path, 'w') as json_file:
        json.dump(json_entry, json_file, indent=2)

def JsonToCsv(json_entry, csv_file_path):
     # Open CSV file for writing
    with open(csv_file_path, 'w', newline='') as csv_file:
        # Create a CSV writer
        csv_writer = csv.writer(csv_file)

        # Write header row based on the keys of the second JSON object (assuming at least two objects are present)
        if len(json_entry) >= 2:
            header = list(json_entry[1].keys())
            csv_writer.writerow(header)

            # Write data rows starting from the second object
            for entry in json_entry[1:]:
                csv_writer.writerow(entry.values())

In [11]:
# sender_side_file
sent_raw_path = path + "raw/" + sent_file_name
sent_qlog_file_path = sent_raw_path + ".qlog"
sent_json_file_path = sent_raw_path + ".json"
sent_csv_file_path = sent_raw_path + ".csv"
sent_json_entry = QlogToJsonEntry(sent_qlog_file_path)
# QlogToJson(sent_json_entry, sent_json_file_path)
JsonToCsv(sent_json_entry, sent_csv_file_path)

In [12]:
received_raw_path = path + "raw/" + received_file_name
received_qlog_file_path = received_raw_path + ".qlog"
received_json_file_path = received_raw_path + ".json"
received_csv_file_path = received_raw_path + ".csv"
received_json_entry = QlogToJsonEntry(received_qlog_file_path)
# QlogToJson(received_json_entry, received_json_file_path)
JsonToCsv(received_json_entry, received_csv_file_path)

In [13]:
sent_df = pd.read_csv(sent_csv_file_path)
received_df = pd.read_csv(received_csv_file_path)

Set time to UMT+8.

In [14]:
def GetStartTime(json_data):
    # unit: ms
    refTime = json_data[0]["trace"]["common_fields"]["reference_time"]
    return refTime

def ProcessTime(df, reference_time):
    # Extract the "time" values from the DataFrame
    original_times = (df['time'].astype(float))

    # Calculate "epoch_time" and convert to timestamps
    epoch_times = (reference_time + original_times)
    timestamps = pd.to_datetime(epoch_times, unit='ms').dt.strftime('%Y-%m-%d %H:%M:%S.%f')

    df['epoch_time'] = epoch_times
    df['timestamp'] = timestamps

    return df

In [15]:
# No matter downlink or uplink, the file time that need to change is client side.
if int(port)%2 == 0: # UL
    clientStartTime = GetStartTime(sent_json_entry)
    print(clientStartTime)
    serverStartTime = GetStartTime(received_json_entry)
    print(serverStartTime)

    senderRefTime = clientStartTime + mean_diff
    rcverRefTime = serverStartTime

else:   # DL
    clientStartTime = GetStartTime(received_json_entry)
    print(clientStartTime)
    serverStartTime = GetStartTime(sent_json_entry)
    print(serverStartTime)
    startTimeDiff = (clientStartTime - serverStartTime) + mean_diff

    senderRefTime = serverStartTime
    rcverRefTime = clientStartTime + mean_diff


1712816948210.7795
1712816893087.4956


In [16]:
sent_df = ProcessTime(sent_df, senderRefTime)
# Add 8 hours to both epoch times and timestamps to match UMT+8
# Also sync time with server
epoch_times_gmt8 = sent_df["epoch_time"] + 8 * 3600 * 1000
sent_df["epoch_time"] = epoch_times_gmt8
timestamps_gmt8 = pd.to_datetime(epoch_times_gmt8, unit='ms').dt.strftime('%Y-%m-%d %H:%M:%S.%f')
sent_df["timestamp"] = timestamps_gmt8

sent_df[-5:]

,time,name,data,epoch_time,timestamp
2857836,450088.208630,recovery:loss_timer_updated,{'event_type': 'cancelled'},1.712846e+12,2024-04-11 14:35:43.137810
2857837,450088.213474,transport:packet_received,"{'header': {'packet_type': '1RTT', 'packet_num...",1.712846e+12,2024-04-11 14:35:43.137815
2857838,450107.523005,transport:packet_sent,"{'header': {'packet_type': '1RTT', 'dcid': '03...",1.712846e+12,2024-04-11 14:35:43.157125
2857839,451065.241963,transport:connection_closed,"{'owner': 'local', 'application_code': 0, 'rea...",1.712846e+12,2024-04-11 14:35:44.114844
2857840,451065.392328,transport:packet_sent,"{'header': {'packet_type': '1RTT', 'dcid': '03...",1.712846e+12,2024-04-11 14:35:44.114994


In [17]:
received_df = ProcessTime(received_df, rcverRefTime)
# if the sender is server, then it is no need to calculate time difference
epoch_times_gmt8 = received_df["epoch_time"] + 8 * 3600 * 1000
received_df["epoch_time"] = epoch_times_gmt8
timestamps_gmt8 = pd.to_datetime(epoch_times_gmt8, unit='ms').dt.strftime('%Y-%m-%d %H:%M:%S.%f')
received_df["timestamp"] = timestamps_gmt8

received_df[-5:]

,time,name,data,epoch_time,timestamp
1132822,450075.955948,recovery:metrics_updated,{'bytes_in_flight': 0},1.712846e+12,2024-04-11 14:35:43.163451
1132823,450075.961605,recovery:loss_timer_updated,{'event_type': 'cancelled'},1.712846e+12,2024-04-11 14:35:43.163457
1132824,450075.966913,transport:packet_received,"{'header': {'packet_type': '1RTT', 'packet_num...",1.712846e+12,2024-04-11 14:35:43.163462
1132825,451053.365834,transport:packet_received,"{'header': {'packet_type': '1RTT', 'packet_num...",1.712846e+12,2024-04-11 14:35:44.140861
1132826,451053.420870,transport:connection_closed,"{'owner': 'remote', 'application_code': 0, 're...",1.712846e+12,2024-04-11 14:35:44.140916


In [18]:
## Add RealTime to CSV
new_csv_order = ['time', 'epoch_time', 'timestamp', 'name', 'data']
sent_df = sent_df[new_csv_order]
received_df = received_df[new_csv_order]

sent_df.to_csv(sent_csv_file_path, index=False)
received_df.to_csv(received_csv_file_path, index=False)

Parse the data.

In [100]:
# sender side data
metrics_all_rows = sent_df[(sent_df['name'] == 'recovery:metrics_updated') & (sent_df['data'].str.contains("'bytes_in_flight':"))]
metrics_sent_rows = sent_df[(sent_df['name'] == 'recovery:metrics_updated') & (sent_df['data'].str.contains("{'bytes_in_flight':"))]
metrics_ack_rows = sent_df[(sent_df['name'] == 'recovery:metrics_updated') & (sent_df['data'].str.contains("'latest_rtt':"))]
total_sent_rows = sent_df[(sent_df['name'] == 'transport:packet_sent')]
pk_sent_rows = sent_df[(sent_df['name'] == 'transport:packet_sent') & (sent_df['data'].str.contains("'frame_type': 'stream'"))]
rcv_ack_rows = sent_df[(sent_df['name'] == 'transport:packet_received') & (sent_df['data'].str.contains("'frame_type': 'ack'")) & (sent_df['data'].str.contains("'packet_type': '1RTT'"))]
lost_rows = sent_df[sent_df['name'] == 'recovery:packet_lost']

# Get the count of rows
metrics_all_cnt = len(metrics_all_rows)
metrics_c_cnt = len(metrics_sent_rows)
metrics_ack_cnt = len(metrics_ack_rows)
total_sent_cnt = len(total_sent_rows)
pk_sent_cnt = len(pk_sent_rows)
rcv_ack_cnt = len(rcv_ack_rows)
lost_cnt = len(lost_rows)

print("packet_sent: ", pk_sent_cnt, metrics_c_cnt)
print("ack: ", rcv_ack_cnt, metrics_ack_cnt)
print(metrics_all_cnt, metrics_c_cnt, metrics_ack_cnt, pk_sent_cnt, rcv_ack_cnt, lost_cnt)

packet_sent:  1409661 1409698
ack:  121216 120771
1530470 1409698 120771 1409661 121216 1249


In [101]:
pk_rcv_rows = received_df[(received_df['name'] == "transport:packet_received") & (received_df['data'].str.contains("'frame_type': 'stream'"))]
pk_rcv_rows = pk_rcv_rows.reset_index(drop=True)
print(len(pk_rcv_rows))
pk_rcv_rows[:5]

1408675


,time,name,data,epoch_time,timestamp
0,703.429062,transport:packet_received,"{'header': {'packet_type': '1RTT', 'packet_num...",1.711036e+12,2024-03-21 15:39:25.242569
1,703.445104,transport:packet_received,"{'header': {'packet_type': '1RTT', 'packet_num...",1.711036e+12,2024-03-21 15:39:25.242585
2,703.457395,transport:packet_received,"{'header': {'packet_type': '1RTT', 'packet_num...",1.711036e+12,2024-03-21 15:39:25.242597
3,703.485729,transport:packet_received,"{'header': {'packet_type': '1RTT', 'packet_num...",1.711036e+12,2024-03-21 15:39:25.242626
4,703.736979,transport:packet_received,"{'header': {'packet_type': '1RTT', 'packet_num...",1.711036e+12,2024-03-21 15:39:25.242877


## Deal with sender side data
Concat `transport:packet_sent` & `recovery:metrics_updated`.

In [102]:
metrics_sent_csv_file_path = path + "middle/" + f"sent_metrics_{time}_{port}.csv"
metrics_sent_rows.to_csv(metrics_sent_csv_file_path, index=False)
pk_sent_csv_file_path = path + "middle/" + f"pk_sent_{time}_{port}.csv"
pk_sent_rows.to_csv(pk_sent_csv_file_path, index=False)

In [103]:
def insert(df, idx, new_row):
    df1 = df.iloc[:idx, :]
    df2 = df.iloc[idx:, :]
    df_new = pd.concat([df1, new_row, df2], ignore_index=True)
    return df_new

In [104]:
metrics_sent_rows = metrics_sent_rows.reset_index(drop=True)
pk_sent_rows = pk_sent_rows.reset_index(drop=True)
print(metrics_sent_rows[:5])
print(pk_sent_rows[:5])


        time                      name  \
0   1.906961  recovery:metrics_updated   
1   1.911361  recovery:metrics_updated   
2  50.677372  recovery:metrics_updated   
3  51.008004  recovery:metrics_updated   
4  51.189732  recovery:metrics_updated   

                                                data    epoch_time  \
0  {'bytes_in_flight': 1161, 'packets_in_flight': 2}  1.711036e+12   
1  {'bytes_in_flight': 1252, 'packets_in_flight': 3}  1.711036e+12   
2   {'bytes_in_flight': 812, 'packets_in_flight': 2}  1.711036e+12   
3   {'bytes_in_flight': 304, 'packets_in_flight': 1}  1.711036e+12   
4  {'bytes_in_flight': 1325, 'packets_in_flight': 2}  1.711036e+12   

                    timestamp  
0  2024-03-21 15:39:25.187278  
1  2024-03-21 15:39:25.187283  
2  2024-03-21 15:39:25.236049  
3  2024-03-21 15:39:25.236379  
4  2024-03-21 15:39:25.236561  
        time                   name  \
0  51.180653  transport:packet_sent   
1  51.204120  transport:packet_sent   
2  51.230799  tra

In [105]:
ori_recover_c_len = len(metrics_sent_rows)
for i in range(pk_sent_cnt):
    if(i >= len(metrics_sent_rows)):
        data = metrics_sent_rows.iloc[i-1]['data']
        new_row_data = {'time': [pk_sent_rows.iloc[i]['time']], 'name':['recovery:metrics_updated'], 'data': [data]}
        new_row = pd.DataFrame(new_row_data)
        metrics_sent_rows = pd.concat([metrics_sent_rows, new_row], ignore_index=True)
        continue
    time_diff = metrics_sent_rows.iloc[i]['time'] - pk_sent_rows.iloc[i]['time']
    # print(i, time_diff)
    # time_diff >= 1: not the matching metrics_update
    while time_diff >= 1:
        data = metrics_sent_rows.iloc[i-1]['data']
        new_row_data = {'time': [pk_sent_rows.iloc[i]['time']], 'name':['recovery:metrics_updated'], 'data': [data]}
        new_row = pd.DataFrame(new_row_data)
        # print(new_row)
        metrics_sent_rows = insert(metrics_sent_rows, i, new_row)
        time_diff = metrics_sent_rows.iloc[i]['time'] - pk_sent_rows.iloc[i]['time']
    # time_diff < 0: missing metrics_update
    while time_diff < 0:
        # print(i, time_diff_list)
        metrics_sent_rows.drop(index=metrics_sent_rows.index[i], inplace=True)
        time_diff = metrics_sent_rows.iloc[i]['time'] - pk_sent_rows.iloc[i]['time']

    

# if len(metrics_sent_rows) < pk_sent_cnt:
#     d = pk_sent_cnt - len(metrics_sent_rows)
# data = metrics_sent_rows.iloc[len(metrics_sent_rows)-1]['data']

# for i in range(d):
#     last_row_data = {'time': [pk_sent_rows.iloc[len(metrics_sent_rows)-1]['time']], 'name':['recovery:metrics_updated'], 'data': [data]}
#     new_row_df = pd.DataFrame(last_row_data)
#     metrics_sent_rows = pd.concat([metrics_sent_rows, new_row], ignore_index=True)

print(ori_recover_c_len, len(metrics_sent_rows))


1409698 1409661


In [106]:
metrics_sent_rows = metrics_sent_rows.reset_index(drop=True)
pk_sent_rows = pk_sent_rows.reset_index(drop=True)
print(len(metrics_sent_rows), len(pk_sent_rows))

# check whether there's still mismatch exist.
time_diff_list = metrics_sent_rows['time'] - pk_sent_rows['time']
mismatch_indices = time_diff_list[(time_diff_list >= 1) | (time_diff_list < 0)].index
if len(mismatch_indices) == 0:
    print("All Matched!")
else:
    print(mismatch_indices)


1409661 1409661
All Matched!


In [107]:
# extract bytes_in_flight & packets_in_flight
metrics_sent_rows['bytes_in_flight'] = None
metrics_sent_rows['packets_in_flight'] = None

# Use ast.literal_eval to safely evaluate the string and extract 'bytes_in_flight' and 'packets_in_flight'
metrics_sent_rows[['bytes_in_flight', 'packets_in_flight']] = metrics_sent_rows['data'].apply(
    lambda x: pd.Series(ast.literal_eval(x)) if isinstance(x, str) else pd.Series([None, None]))

metrics_sent_rows[:5]

,time,name,data,epoch_time,timestamp,bytes_in_flight,packets_in_flight
0,51.189732,recovery:metrics_updated,"{'bytes_in_flight': 1325, 'packets_in_flight': 2}",1.711036e+12,2024-03-21 15:39:25.236561,1325.0,2.0
1,51.208380,recovery:metrics_updated,"{'bytes_in_flight': 2348, 'packets_in_flight': 3}",1.711036e+12,2024-03-21 15:39:25.236580,2348.0,3.0
2,51.235548,recovery:metrics_updated,"{'bytes_in_flight': 3371, 'packets_in_flight': 4}",1.711036e+12,2024-03-21 15:39:25.236607,3371.0,4.0
3,51.547113,recovery:metrics_updated,"{'bytes_in_flight': 4394, 'packets_in_flight': 5}",1.711036e+12,2024-03-21 15:39:25.236919,4394.0,5.0
4,51.564154,recovery:metrics_updated,"{'bytes_in_flight': 5417, 'packets_in_flight': 6}",1.711036e+12,2024-03-21 15:39:25.236935,5417.0,6.0


In [108]:
# Add bytes_in_flight & packets_in_flight to pk_sent_rows
pk_sent_rows['bytes_in_flight'] = metrics_sent_rows['bytes_in_flight']
pk_sent_rows['packets_in_flight'] = metrics_sent_rows['packets_in_flight']

pk_sent_rows[:5]

,time,name,data,epoch_time,timestamp,bytes_in_flight,packets_in_flight
0,51.180653,transport:packet_sent,"{'header': {'packet_type': '1RTT', 'packet_num...",1.711036e+12,2024-03-21 15:39:25.236552,1325.0,2.0
1,51.204120,transport:packet_sent,"{'header': {'packet_type': '1RTT', 'packet_num...",1.711036e+12,2024-03-21 15:39:25.236576,2348.0,3.0
2,51.230799,transport:packet_sent,"{'header': {'packet_type': '1RTT', 'packet_num...",1.711036e+12,2024-03-21 15:39:25.236602,3371.0,4.0
3,51.538941,transport:packet_sent,"{'header': {'packet_type': '1RTT', 'packet_num...",1.711036e+12,2024-03-21 15:39:25.236910,4394.0,5.0
4,51.558916,transport:packet_sent,"{'header': {'packet_type': '1RTT', 'packet_num...",1.711036e+12,2024-03-21 15:39:25.236930,5417.0,6.0


Concat `transport:packet_received` & `recovery:metrics_updated`.

In [109]:
metrics_ack_csv_file_path = path + "middle/" + f"ack_metrics_{time}_{port}.csv" 
metrics_ack_rows.to_csv(metrics_ack_csv_file_path, index=False)
rcv_ack_csv_file_path = path + "middle/" + f"rcv_ack_{time}_{port}.csv"
rcv_ack_rows.to_csv(rcv_ack_csv_file_path, index=False)

In [110]:
metrics_ack_rows = metrics_ack_rows.reset_index(drop=True)
rcv_ack_rows = rcv_ack_rows.reset_index(drop=True)
initial_ack_metrics = metrics_ack_rows.iloc[[0]]
metrics_ack_rows.drop(index=metrics_ack_rows.index[0], inplace=True)
metrics_ack_rows = metrics_ack_rows.reset_index(drop=True)

In [111]:
metrics_ack_rows[:3]

,time,name,data,epoch_time,timestamp
0,50.945984,recovery:metrics_updated,"{'min_rtt': 48.780818, 'smoothed_rtt': 48.7808...",1.711036e+12,2024-03-21 15:39:25.236317
1,92.646097,recovery:metrics_updated,"{'min_rtt': 40.934088, 'smoothed_rtt': 47.799,...",1.711036e+12,2024-03-21 15:39:25.278017
2,103.715050,recovery:metrics_updated,"{'smoothed_rtt': 48.262, 'latest_rtt': 51.5058...",1.711036e+12,2024-03-21 15:39:25.289086


In [112]:
rcv_ack_rows[:3]

,time,name,data,epoch_time,timestamp
0,50.954086,transport:packet_received,"{'header': {'packet_type': '1RTT', 'packet_num...",1.711036e+12,2024-03-21 15:39:25.236325
1,92.654129,transport:packet_received,"{'header': {'packet_type': '1RTT', 'packet_num...",1.711036e+12,2024-03-21 15:39:25.278025
2,103.731113,transport:packet_received,"{'header': {'packet_type': '1RTT', 'packet_num...",1.711036e+12,2024-03-21 15:39:25.289103


In [113]:
print(len(metrics_ack_rows), len(rcv_ack_rows))
for i in range(rcv_ack_cnt):
    if(i >= len(metrics_ack_rows)):
        data = metrics_ack_rows.iloc[i-1]['data']
        new_row_data = {'time': [rcv_ack_rows.iloc[i]['time']], 'name':['recovery:metrics_updated'], 'data': [data]}
        new_row = pd.DataFrame(new_row_data)
        metrics_ack_rows = pd.concat([metrics_ack_rows, new_row], ignore_index=True)
        continue
    time_diff = metrics_ack_rows.iloc[i]['time'] - rcv_ack_rows.iloc[i]['time']
    # time_diff >= 1: not the matching metrics_update
    while time_diff > 0:
        # print("> 0:", i, time_diff)
        if i == 0:
            data = initial_ack_metrics.iloc[0]['data']
        else:
            data = metrics_ack_rows.iloc[i-1]['data']
        new_row_data = {'time': [rcv_ack_rows.iloc[i]['time']], 'name':['recovery:metrics_updated'], 'data': [data]}
        new_row = pd.DataFrame(new_row_data)
        metrics_ack_rows = insert(metrics_ack_rows, i, new_row)
        time_diff = metrics_ack_rows.iloc[i]['time'] - rcv_ack_rows.iloc[i]['time']
    # time_diff < 0: missing metrics_update
    while time_diff <= -1:
        # print("<= -1:", i, time_diff)
        metrics_ack_rows.drop(index=metrics_ack_rows.index[i], inplace=True)
        time_diff = metrics_ack_rows.iloc[i]['time'] - rcv_ack_rows.iloc[i]['time']
print(len(metrics_ack_rows), len(rcv_ack_rows))

120770 121216
121216 121216


In [114]:
metrics_ack_rows = metrics_ack_rows.reset_index(drop=True)
rcv_ack_rows = rcv_ack_rows.reset_index(drop=True)

# check whether there's still mismatch exist.
time_diff_list = metrics_ack_rows['time'] - rcv_ack_rows['time']
mismatch_indices = time_diff_list[(time_diff_list <= -1) | (time_diff_list > 0)].index
if len(mismatch_indices) == 0:
    print("All Matched!")
else:
    print(mismatch_indices)

Index([51710], dtype='int64')


In [115]:
ack_json_list = []
## Add the initial_ack_metrics for temporary
print(initial_ack_metrics)
metrics_ack_rows = pd.concat([initial_ack_metrics, metrics_ack_rows], axis=0).reset_index(drop=True)
for i in range(len(metrics_ack_rows)):
    s = metrics_ack_rows.iloc[i]['data'].replace("\'", "\"")
    json_object = json.loads(s)
    ack_json_list.append(json_object)

metrics_ack_df = pd.DataFrame(ack_json_list)
# Fill missing values in each row with the previous row's values
metrics_ack_df = metrics_ack_df.ffill(axis=0)

## drop initial_ack_metrics
metrics_ack_rows.drop(index=metrics_ack_rows.index[0], inplace=True)
metrics_ack_rows = metrics_ack_rows.reset_index(drop=True)
metrics_ack_df.drop(index=metrics_ack_df.index[0], inplace=True)
metrics_ack_df = metrics_ack_df.reset_index(drop=True)
metrics_ack_df[:5]

       time                      name  \
0  1.898859  recovery:metrics_updated   

                                                data    epoch_time  \
0  {'min_rtt': 0, 'smoothed_rtt': 0, 'latest_rtt'...  1.711036e+12   

                    timestamp  
0  2024-03-21 15:39:25.187270  


,min_rtt,smoothed_rtt,latest_rtt,rtt_variance,congestion_window,bytes_in_flight,packets_in_flight
0,48.780818,48.780818,48.780818,24.390409,40064.0,0,1.0
1,40.934088,47.799000,40.934088,20.254000,50080.0,32780,32.0
2,40.934088,48.262000,51.505800,16.117000,55088.0,46113,45.0
3,40.934088,48.521000,50.340909,12.607000,57592.0,44067,43.0
4,40.934088,48.829000,50.989491,10.072000,60096.0,55346,54.0


In [116]:
metrics_ack_rows = pd.concat([metrics_ack_rows, metrics_ack_df], axis=1).reset_index(drop=True)
# since we have parse out all the information in data, we can drop the data cl=olumn
metrics_ack_rows = metrics_ack_rows.drop(columns=['data'])
metrics_ack_rows[:5]

,time,name,epoch_time,timestamp,min_rtt,smoothed_rtt,latest_rtt,rtt_variance,congestion_window,bytes_in_flight,packets_in_flight
0,50.945984,recovery:metrics_updated,1.711036e+12,2024-03-21 15:39:25.236317,48.780818,48.780818,48.780818,24.390409,40064.0,0,1.0
1,92.646097,recovery:metrics_updated,1.711036e+12,2024-03-21 15:39:25.278017,40.934088,47.799000,40.934088,20.254000,50080.0,32780,32.0
2,103.715050,recovery:metrics_updated,1.711036e+12,2024-03-21 15:39:25.289086,40.934088,48.262000,51.505800,16.117000,55088.0,46113,45.0
3,103.740752,recovery:metrics_updated,1.711036e+12,2024-03-21 15:39:25.289112,40.934088,48.521000,50.340909,12.607000,57592.0,44067,43.0
4,105.797663,recovery:metrics_updated,1.711036e+12,2024-03-21 15:39:25.291169,40.934088,48.829000,50.989491,10.072000,60096.0,55346,54.0


In [117]:
# Check whehter the length is equal before concating metrics into rcv_ack_rows
print(len(rcv_ack_rows), len(metrics_ack_df))

121216 121216


In [118]:
rcv_ack_rows = pd.concat([rcv_ack_rows, metrics_ack_df], axis=1)
rcv_ack_rows = rcv_ack_rows.reset_index(drop=True)

print(len(rcv_ack_rows), len(metrics_ack_df))
rcv_ack_rows[-5:]

121216 121216


,time,name,data,epoch_time,timestamp,min_rtt,smoothed_rtt,latest_rtt,rtt_variance,congestion_window,bytes_in_flight,packets_in_flight
121211,450068.519355,transport:packet_received,"{'header': {'packet_type': '1RTT', 'packet_num...",1.711036e+12,2024-03-21 15:46:55.253891,10.717831,27.246,25.483790,2.476,152124.0,42189,41.0
121212,450068.535140,transport:packet_received,"{'header': {'packet_type': '1RTT', 'packet_num...",1.711036e+12,2024-03-21 15:46:55.253906,10.717831,26.770,23.440854,2.808,152124.0,33957,33.0
121213,450075.257745,transport:packet_received,"{'header': {'packet_type': '1RTT', 'packet_num...",1.711036e+12,2024-03-21 15:46:55.260629,10.717831,26.881,27.662282,2.329,152124.0,20580,20.0
121214,450075.282120,transport:packet_received,"{'header': {'packet_type': '1RTT', 'packet_num...",1.711036e+12,2024-03-21 15:46:55.260654,10.717831,26.720,25.597859,2.067,152124.0,3087,3.0
121215,450075.297275,transport:packet_received,"{'header': {'packet_type': '1RTT', 'packet_num...",1.711036e+12,2024-03-21 15:46:55.260669,10.717831,25.808,19.431836,3.372,152124.0,0,3.0


Mapping the ACK ranges

In [119]:
acked_ranges_series = rcv_ack_rows['data']
acked_ranges_list = []
for i in range(len(acked_ranges_series)):
    s = acked_ranges_series.iloc[i]
    data_dict = json.loads(s.replace("\'", "\""))
    # Extract 'acked_ranges' from all frames
    acked_ranges = [range_entry for frame in data_dict['frames'] if 'acked_ranges' in frame for range_entry in frame['acked_ranges']]
    acked_ranges_list.append(acked_ranges)

acked_ranges_df = pd.DataFrame({"acked_ranges": acked_ranges_list})
acked_ranges_df[:5]

,acked_ranges
0,[[0]]
1,"[[0, 8]]"
2,"[[0, 12]]"
3,"[[0, 14]]"
4,"[[0, 16]]"


In [120]:
rcv_ack_rows = pd.concat([rcv_ack_rows, acked_ranges_df], axis=1)
rcv_ack_rows = rcv_ack_rows.reset_index(drop=True)

rcv_ack_rows[:5]

,time,name,data,epoch_time,timestamp,min_rtt,smoothed_rtt,latest_rtt,rtt_variance,congestion_window,bytes_in_flight,packets_in_flight,acked_ranges
0,50.954086,transport:packet_received,"{'header': {'packet_type': '1RTT', 'packet_num...",1.711036e+12,2024-03-21 15:39:25.236325,48.780818,48.780818,48.780818,24.390409,40064.0,0,1.0,[[0]]
1,92.654129,transport:packet_received,"{'header': {'packet_type': '1RTT', 'packet_num...",1.711036e+12,2024-03-21 15:39:25.278025,40.934088,47.799000,40.934088,20.254000,50080.0,32780,32.0,"[[0, 8]]"
2,103.731113,transport:packet_received,"{'header': {'packet_type': '1RTT', 'packet_num...",1.711036e+12,2024-03-21 15:39:25.289103,40.934088,48.262000,51.505800,16.117000,55088.0,46113,45.0,"[[0, 12]]"
3,103.746898,transport:packet_received,"{'header': {'packet_type': '1RTT', 'packet_num...",1.711036e+12,2024-03-21 15:39:25.289118,40.934088,48.521000,50.340909,12.607000,57592.0,44067,43.0,"[[0, 14]]"
4,105.810864,transport:packet_received,"{'header': {'packet_type': '1RTT', 'packet_num...",1.711036e+12,2024-03-21 15:39:25.291182,40.934088,48.829000,50.989491,10.072000,60096.0,55346,54.0,"[[0, 16]]"


In [121]:
# parse out the packet_number & offset & length
pk_sent_series =  pk_sent_rows['data']
pk_num_list = []
offset_list = []
length_list = []
for i in range(len(pk_sent_series)):
    s = pk_sent_series.iloc[i]
    data_dict = json.loads(s.replace("\'", "\""))
    packet_number = data_dict['header']['packet_number']
    # Initialize offset to None in case 'frame_type': 'stream' is not found
    offset = None
    # Iterate through frames to find 'offset' for 'frame_type': 'stream'
    for frame in data_dict.get('frames', []):
        if frame.get('frame_type') == 'stream':
            offset = frame.get('offset')
            length = frame.get('length')
            break  # Stop iterating once 'offset' is found
    
    pk_num_list.append(packet_number)
    offset_list.append(offset)
    length_list.append(length)

pk_num_df = pd.DataFrame({"packet_number": pk_num_list, "offset": offset_list, "length": length_list})
pk_num_df[:5]

,packet_number,offset,length
0,2,0,1000
1,3,1000,1000
2,4,2000,1000
3,5,3000,1000
4,6,4000,1000


In [122]:
pk_sent_rows = pd.concat([pk_sent_rows, pk_num_df], axis=1)
pk_sent_rows = pk_sent_rows.reset_index(drop=True)

pk_sent_rows[:5]

,time,name,data,epoch_time,timestamp,bytes_in_flight,packets_in_flight,packet_number,offset,length
0,51.180653,transport:packet_sent,"{'header': {'packet_type': '1RTT', 'packet_num...",1.711036e+12,2024-03-21 15:39:25.236552,1325.0,2.0,2,0,1000
1,51.204120,transport:packet_sent,"{'header': {'packet_type': '1RTT', 'packet_num...",1.711036e+12,2024-03-21 15:39:25.236576,2348.0,3.0,3,1000,1000
2,51.230799,transport:packet_sent,"{'header': {'packet_type': '1RTT', 'packet_num...",1.711036e+12,2024-03-21 15:39:25.236602,3371.0,4.0,4,2000,1000
3,51.538941,transport:packet_sent,"{'header': {'packet_type': '1RTT', 'packet_num...",1.711036e+12,2024-03-21 15:39:25.236910,4394.0,5.0,5,3000,1000
4,51.558916,transport:packet_sent,"{'header': {'packet_type': '1RTT', 'packet_num...",1.711036e+12,2024-03-21 15:39:25.236930,5417.0,6.0,6,4000,1000


In [123]:
pk_sent_rows['smoothed_rtt'] = np.nan
pk_sent_rows['latest_rtt'] = np.nan
pk_sent_rows['rtt_variance'] = np.nan
pk_sent_rows['congestion_window'] = np.nan

pk_sent_rows[:5]

,time,name,data,epoch_time,timestamp,bytes_in_flight,packets_in_flight,packet_number,offset,length,smoothed_rtt,latest_rtt,rtt_variance,congestion_window
0,51.180653,transport:packet_sent,"{'header': {'packet_type': '1RTT', 'packet_num...",1.711036e+12,2024-03-21 15:39:25.236552,1325.0,2.0,2,0,1000,NaN,NaN,NaN,NaN
1,51.204120,transport:packet_sent,"{'header': {'packet_type': '1RTT', 'packet_num...",1.711036e+12,2024-03-21 15:39:25.236576,2348.0,3.0,3,1000,1000,NaN,NaN,NaN,NaN
2,51.230799,transport:packet_sent,"{'header': {'packet_type': '1RTT', 'packet_num...",1.711036e+12,2024-03-21 15:39:25.236602,3371.0,4.0,4,2000,1000,NaN,NaN,NaN,NaN
3,51.538941,transport:packet_sent,"{'header': {'packet_type': '1RTT', 'packet_num...",1.711036e+12,2024-03-21 15:39:25.236910,4394.0,5.0,5,3000,1000,NaN,NaN,NaN,NaN
4,51.558916,transport:packet_sent,"{'header': {'packet_type': '1RTT', 'packet_num...",1.711036e+12,2024-03-21 15:39:25.236930,5417.0,6.0,6,4000,1000,NaN,NaN,NaN,NaN


In [124]:
def update_pk_sent_rows(row):
    acked_ranges = row['acked_ranges']
    smoothed_rtt = row['smoothed_rtt']
    latest_rtt = row['latest_rtt']
    rtt_variance = row['rtt_variance']
    congestion_window = row['congestion_window']

    for ack_range in acked_ranges:
        start_packet, end_packet = ack_range[0], ack_range[-1]
        existing_packets = set(pk_sent_rows['packet_number'])
        packet_numbers_to_update = set(range(start_packet, end_packet + 1)).intersection(existing_packets)

        mask = pk_sent_rows['packet_number'].isin(packet_numbers_to_update)
        pk_sent_rows.loc[mask, 'smoothed_rtt'] = pk_sent_rows.loc[mask, 'smoothed_rtt'].fillna(smoothed_rtt)
        pk_sent_rows.loc[mask, 'latest_rtt'] = pk_sent_rows.loc[mask, 'latest_rtt'].fillna(latest_rtt)
        pk_sent_rows.loc[mask, 'congestion_window'] = pk_sent_rows.loc[mask, 'congestion_window'].fillna(congestion_window)
        pk_sent_rows.loc[mask, 'rtt_variance'] = pk_sent_rows.loc[mask, 'rtt_variance'].fillna(rtt_variance)

# Apply the custom update function to each row in rcv_ack_rows
rcv_ack_rows.apply(update_pk_sent_rows, axis=1)

# Display the updated pk_sent_rows
pk_sent_rows[:5]

,time,name,data,epoch_time,timestamp,bytes_in_flight,packets_in_flight,packet_number,offset,length,smoothed_rtt,latest_rtt,rtt_variance,congestion_window
0,51.180653,transport:packet_sent,"{'header': {'packet_type': '1RTT', 'packet_num...",1.711036e+12,2024-03-21 15:39:25.236552,1325.0,2.0,2,0,1000,47.799,40.934088,20.254,50080.0
1,51.204120,transport:packet_sent,"{'header': {'packet_type': '1RTT', 'packet_num...",1.711036e+12,2024-03-21 15:39:25.236576,2348.0,3.0,3,1000,1000,47.799,40.934088,20.254,50080.0
2,51.230799,transport:packet_sent,"{'header': {'packet_type': '1RTT', 'packet_num...",1.711036e+12,2024-03-21 15:39:25.236602,3371.0,4.0,4,2000,1000,47.799,40.934088,20.254,50080.0
3,51.538941,transport:packet_sent,"{'header': {'packet_type': '1RTT', 'packet_num...",1.711036e+12,2024-03-21 15:39:25.236910,4394.0,5.0,5,3000,1000,47.799,40.934088,20.254,50080.0
4,51.558916,transport:packet_sent,"{'header': {'packet_type': '1RTT', 'packet_num...",1.711036e+12,2024-03-21 15:39:25.236930,5417.0,6.0,6,4000,1000,47.799,40.934088,20.254,50080.0


Identify lost packets

In [125]:
# Use ast.literal_eval to safely evaluate the string and extract 'packet_number'
lost_rows['packet_number'] = lost_rows['data'].apply(lambda x: ast.literal_eval(x)['header']['packet_number'] if isinstance(x, str) else None)
lost_rows['trigger'] = lost_rows['data'].apply(lambda x: ast.literal_eval(x)['trigger'] if isinstance(x, str) else None)
lost_rows[:5]

/var/folders/ll/z00qrhv13397cv4_52zrrb200000gn/T/ipykernel_87570/2583690328.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lost_rows['packet_number'] = lost_rows['data'].apply(lambda x: ast.literal_eval(x)['header']['packet_number'] if isinstance(x, str) else None)
/var/folders/ll/z00qrhv13397cv4_52zrrb200000gn/T/ipykernel_87570/2583690328.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lost_rows['trigger'] = lost_rows['data'].apply(lambda x: ast.literal_eval(x)['trigger'] if isinstance(x, str) els

,time,name,data,epoch_time,timestamp,packet_number,trigger
10846,759.453722,recovery:packet_lost,"{'header': {'packet_type': '1RTT', 'packet_num...",1.711036e+12,2024-03-21 15:39:25.944825,3636,time_threshold
10856,810.613422,recovery:packet_lost,"{'header': {'packet_type': '1RTT', 'packet_num...",1.711036e+12,2024-03-21 15:39:25.995985,3906,time_threshold
10857,810.618660,recovery:packet_lost,"{'header': {'packet_type': '1RTT', 'packet_num...",1.711036e+12,2024-03-21 15:39:25.995990,3907,time_threshold
10858,810.622082,recovery:packet_lost,"{'header': {'packet_type': '1RTT', 'packet_num...",1.711036e+12,2024-03-21 15:39:25.995993,3908,time_threshold
10859,810.625924,recovery:packet_lost,"{'header': {'packet_type': '1RTT', 'packet_num...",1.711036e+12,2024-03-21 15:39:25.995997,3909,time_threshold


In [126]:
lost_pk_csv_file_path = path + "middle/" + f"lost_pk_{time}_{port}.csv"
lost_rows.to_csv(lost_pk_csv_file_path, index=False)

In [127]:
## set to True if the packet is lost
pk_sent_rows['packet_lost'] = False

# Iterate through rows and set 'packet_lost' to True where 'packet_number' values match
for _, lost_row in lost_rows.iterrows():
    packet_number = lost_row['packet_number']
    
    # Check if 'packet_number' exists in pk_sent_rows
    if packet_number in pk_sent_rows['packet_number'].values:
        pk_sent_rows.loc[pk_sent_rows['packet_number'] == packet_number, 'packet_lost'] = True

pk_sent_rows[19340:19345]

,time,name,data,epoch_time,timestamp,bytes_in_flight,packets_in_flight,packet_number,offset,length,smoothed_rtt,latest_rtt,rtt_variance,congestion_window,packet_lost
19340,3487.619602,transport:packet_sent,"{'header': {'packet_type': '1RTT', 'packet_num...",1.711036e+12,2024-03-21 15:39:28.672991,112756.0,110.0,19353,19220000,1000,33.192,27.047266,12.782,354954.0,False
19341,3487.631615,transport:packet_sent,"{'header': {'packet_type': '1RTT', 'packet_num...",1.711036e+12,2024-03-21 15:39:28.673003,113781.0,111.0,19354,19221000,1000,33.192,27.047266,12.782,354954.0,False
19342,3487.656758,transport:packet_sent,"{'header': {'packet_type': '1RTT', 'packet_num...",1.711036e+12,2024-03-21 15:39:28.673028,114806.0,112.0,19355,19222000,1000,33.192,27.047266,12.782,354954.0,False
19343,3488.816971,transport:packet_sent,"{'header': {'packet_type': '1RTT', 'packet_num...",1.711036e+12,2024-03-21 15:39:28.674188,115831.0,113.0,19356,19223000,1000,33.192,27.047266,12.782,354954.0,False
19344,3488.832197,transport:packet_sent,"{'header': {'packet_type': '1RTT', 'packet_num...",1.711036e+12,2024-03-21 15:39:28.674204,116856.0,114.0,19357,19224000,1000,33.192,27.047266,12.782,354954.0,False


In [128]:
cols = ['time', 'epoch_time', 'timestamp', 'name', 'packet_number', 'offset', 'length', 'bytes_in_flight', 'packets_in_flight', 'smoothed_rtt', 'latest_rtt', 'rtt_variance', 'congestion_window', 'packet_lost', 'data']
processed_df = pk_sent_rows[cols]
processed_df[:5]

,time,epoch_time,timestamp,name,packet_number,offset,length,bytes_in_flight,packets_in_flight,smoothed_rtt,latest_rtt,rtt_variance,congestion_window,packet_lost,data
0,51.180653,1.711036e+12,2024-03-21 15:39:25.236552,transport:packet_sent,2,0,1000,1325.0,2.0,47.799,40.934088,20.254,50080.0,False,"{'header': {'packet_type': '1RTT', 'packet_num..."
1,51.204120,1.711036e+12,2024-03-21 15:39:25.236576,transport:packet_sent,3,1000,1000,2348.0,3.0,47.799,40.934088,20.254,50080.0,False,"{'header': {'packet_type': '1RTT', 'packet_num..."
2,51.230799,1.711036e+12,2024-03-21 15:39:25.236602,transport:packet_sent,4,2000,1000,3371.0,4.0,47.799,40.934088,20.254,50080.0,False,"{'header': {'packet_type': '1RTT', 'packet_num..."
3,51.538941,1.711036e+12,2024-03-21 15:39:25.236910,transport:packet_sent,5,3000,1000,4394.0,5.0,47.799,40.934088,20.254,50080.0,False,"{'header': {'packet_type': '1RTT', 'packet_num..."
4,51.558916,1.711036e+12,2024-03-21 15:39:25.236930,transport:packet_sent,6,4000,1000,5417.0,6.0,47.799,40.934088,20.254,50080.0,False,"{'header': {'packet_type': '1RTT', 'packet_num..."


In [129]:
csv_file_path = path + "data/" + f"processed_sent_{time}_{port}.csv"
processed_df.to_csv(csv_file_path, sep='@', index=False)

## Receiver side data

In [130]:
pk_rcv_df = pk_rcv_rows.reset_index(drop=True)
pk_rcv_df[:5]

,time,name,data,epoch_time,timestamp
0,703.429062,transport:packet_received,"{'header': {'packet_type': '1RTT', 'packet_num...",1.711036e+12,2024-03-21 15:39:25.242569
1,703.445104,transport:packet_received,"{'header': {'packet_type': '1RTT', 'packet_num...",1.711036e+12,2024-03-21 15:39:25.242585
2,703.457395,transport:packet_received,"{'header': {'packet_type': '1RTT', 'packet_num...",1.711036e+12,2024-03-21 15:39:25.242597
3,703.485729,transport:packet_received,"{'header': {'packet_type': '1RTT', 'packet_num...",1.711036e+12,2024-03-21 15:39:25.242626
4,703.736979,transport:packet_received,"{'header': {'packet_type': '1RTT', 'packet_num...",1.711036e+12,2024-03-21 15:39:25.242877


In [131]:
pk_rcv_series =  pk_rcv_df['data']
pk_rcv_num_list = []
offset_rcv_list = []
length_rcv_list = []
for i in range(len(pk_rcv_series)):
    s = pk_rcv_series.iloc[i]
    data_dict = json.loads(s.replace("\'", "\""))
    packet_number = data_dict['header']['packet_number']
    # Initialize offset to None in case 'frame_type': 'stream' is not found
    offset = None
    # Iterate through frames to find 'offset' for 'frame_type': 'stream'
    for frame in data_dict.get('frames', []):
        if frame.get('frame_type') == 'stream':
            offset = frame.get('offset')
            length = frame.get('length')
            break  # Stop iterating once 'offset' is found
    
    pk_rcv_num_list.append(packet_number)
    offset_rcv_list.append(offset)
    length_rcv_list.append(length)

pk_rcv_df['packet_number'] = pk_rcv_num_list
pk_rcv_df['offset'] = offset_rcv_list
pk_rcv_df['length'] = length_rcv_list

pk_rcv_df[:5]

,time,name,data,epoch_time,timestamp,packet_number,offset,length
0,703.429062,transport:packet_received,"{'header': {'packet_type': '1RTT', 'packet_num...",1.711036e+12,2024-03-21 15:39:25.242569,2,0,1000
1,703.445104,transport:packet_received,"{'header': {'packet_type': '1RTT', 'packet_num...",1.711036e+12,2024-03-21 15:39:25.242585,3,1000,1000
2,703.457395,transport:packet_received,"{'header': {'packet_type': '1RTT', 'packet_num...",1.711036e+12,2024-03-21 15:39:25.242597,4,2000,1000
3,703.485729,transport:packet_received,"{'header': {'packet_type': '1RTT', 'packet_num...",1.711036e+12,2024-03-21 15:39:25.242626,5,3000,1000
4,703.736979,transport:packet_received,"{'header': {'packet_type': '1RTT', 'packet_num...",1.711036e+12,2024-03-21 15:39:25.242877,6,4000,1000


In [132]:
cols = ['time', 'epoch_time', 'timestamp', 'name', 'packet_number', 'offset', 'length', 'data']
processed_rcv_df = pk_rcv_df[cols]
processed_rcv_df[:5]


,time,epoch_time,timestamp,name,packet_number,offset,length,data
0,703.429062,1.711036e+12,2024-03-21 15:39:25.242569,transport:packet_received,2,0,1000,"{'header': {'packet_type': '1RTT', 'packet_num..."
1,703.445104,1.711036e+12,2024-03-21 15:39:25.242585,transport:packet_received,3,1000,1000,"{'header': {'packet_type': '1RTT', 'packet_num..."
2,703.457395,1.711036e+12,2024-03-21 15:39:25.242597,transport:packet_received,4,2000,1000,"{'header': {'packet_type': '1RTT', 'packet_num..."
3,703.485729,1.711036e+12,2024-03-21 15:39:25.242626,transport:packet_received,5,3000,1000,"{'header': {'packet_type': '1RTT', 'packet_num..."
4,703.736979,1.711036e+12,2024-03-21 15:39:25.242877,transport:packet_received,6,4000,1000,"{'header': {'packet_type': '1RTT', 'packet_num..."


In [133]:
csv_file_path = path + "data/" + f"processed_rcv_{time}_{port}.csv"
processed_rcv_df.to_csv(csv_file_path, sep='@')

## Real Packet Loss Data

In [154]:
### USER SETTINGS ###
database="/Volumes/mollyT7/MOXA/"
# database = "/Volumes/MOLLY256/MOXA/"
# database = "/Users/molly/Desktop"
dates = [
    "2024-03-13",
    # "2024-03-21",
]
devices = sorted([
    "sm00",
    # "sm01",
])
exps = {
    # "QUIC-450sec": (6, ["#{:02d}".format(i + 1) for i in range(6)]),
    # "QUIC-300sec": (6, ["#{:02d}".format(i + 1) for i in range(6)]),
    "QUIC-1M": (2, ["#{:02d}".format(i + 1) for i in range(2)]),
    "QUIC-5M": (2, ["#{:02d}".format(i + 1) for i in range(2)]),
    "QUIC-10M": (2, ["#{:02d}".format(i + 1) for i in range(2)]),
    # "QUIC-inf": (6, ["#{:02d}".format(i + 1) for i in range(6)]),
}

device_to_port = {"sm00": [5200, 5201], 
                  "sm01": [5202, 5203],
                  "sm02": [5204, 5205]}



In [155]:
def find_ul_sender_file(database, date, exp, device):
    ul_files = []
    exp_rounds, exp_list = exps[exp]
    ports = device_to_port.get(device, [])
    for exp_round in exp_list:
        folder_path = os.path.join(database, date, exp, device, exp_round, 'raw')
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                if "client.csv" in file:
                    # Extract the numbers from the file name
                    numbers = file.split("_")[3]
                    if str(ports[0]) in numbers:
                        ul_files.append(os.path.join(root, file))
                        break  # Exit the inner loop once the port is found
    return ul_files

def find_dl_sender_file(database, date, exp, device):
    dl_files = []
    exp_rounds, exp_list = exps[exp]
    ports = device_to_port.get(device, [])
    for exp_round in exp_list:
        folder_path = os.path.join(database, date, exp, device, exp_round, 'raw')
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                if "server.csv" in file:
                    # Extract the numbers from the file name
                    numbers = file.split("_")[3]
                    if str(ports[1]) in numbers:
                        dl_files.append(os.path.join(root, file))
                        break  # Exit the inner loop once the port is found
    return dl_files

def find_ul_rcv_file(database, date, exp, device):
    ul_files = []
    exp_rounds, exp_list = exps[exp]
    ports = device_to_port.get(device, [])
    for exp_round in exp_list:
        folder_path = os.path.join(database, date, exp, device, exp_round, 'data')
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                if "processed_rcv" in file:
                    # Extract the numbers from the file name
                    numbers = file.split("_")[3]
                    if str(ports[0]) in numbers:
                        ul_files.append(os.path.join(root, file))
                        break  # Exit the inner loop once the port is found
    return ul_files

def find_dl_rcv_file(database, date, exp, device):
    dl_files = []
    exp_rounds, exp_list = exps[exp]
    ports = device_to_port.get(device, [])
    for exp_round in exp_list:
        folder_path = os.path.join(database, date, exp, device, exp_round, 'data')
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                if "processed_rcv" in file:
                    # Extract the numbers from the file name
                    numbers = file.split("_")[3]
                    if str(ports[1]) in numbers:
                        dl_files.append(os.path.join(root, file))
                        break  # Exit the inner loop once the port is found
    return dl_files

def find_ul_loss_file(database, date, exp, device):
    ul_loss_files = []
    exp_rounds, exp_list = exps[exp]
    ports = device_to_port.get(device, [])
    for exp_round in exp_list:
        folder_path = os.path.join(database, date, exp, device, exp_round, 'middle')
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                if "lost_pk" in file:
                    # Extract the numbers from the file name
                    numbers = file.split("_")[3]
                    if str(ports[0]) in numbers:
                        ul_loss_files.append(os.path.join(root, file))
                        break  # Exit the inner loop once the port is found
    return ul_loss_files

def find_dl_loss_file(database, date, exp, device):
    dl_loss_files = []
    exp_rounds, exp_list = exps[exp]
    ports = device_to_port.get(device, [])
    for exp_round in exp_list:
        folder_path = os.path.join(database, date, exp, device, exp_round, 'middle')
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                if "lost_pk" in file:
                    # Extract the numbers from the file name
                    numbers = file.split("_")[3]
                    if str(ports[1]) in numbers:
                        dl_loss_files.append(os.path.join(root, file))
                        break  # Exit the inner loop once the port is found
    return dl_loss_files


In [156]:
def get_loss_data(lost_df, received_df):
    # Check if each row in ul_lost_df['packet_number'] is present in ul_received_df['packet_number']
    lost_in_received = lost_df['packet_number'].isin(received_df['packet_number'])

    # Get the rows in ul_lost_df where the packet number is present in ul_received_df
    exec_lat_df = lost_df[lost_in_received]

    exec_reorder_df = exec_lat_df[exec_lat_df['trigger'] == 'reordering_threshold']
    exec_time_df = exec_lat_df[exec_lat_df['trigger'] == 'time_threshold']

    # Get the rows in ul_lost_df where the packet number is not present in ul_received_df
    real_lost_df = lost_df[~lost_in_received]

    # Filter ul_lost_df for rows where 'trigger' is 'reordering threshold'
    lost_reorder_df = real_lost_df[real_lost_df['trigger'] == 'reordering_threshold']
    lost_time_df = real_lost_df[real_lost_df['trigger'] == 'time_threshold']

    return exec_lat_df, exec_reorder_df, exec_time_df, real_lost_df, lost_reorder_df, lost_time_df

def calculate_statistics(lost_reorder_df, lost_time_df, real_lost_df, exec_reorder_df, exec_time_df, exec_lat_df, lost_df, data_df, sent_df):
    statistics_data = [{
        'total_packets': len(sent_df),
        'total_data_packets': len(data_df),
        'original_pkl': len(lost_df),
        'reordering_threshold': len(lost_reorder_df),
        'time_threshold': len(lost_time_df),
        'real_pkl': len(real_lost_df),
        'exec_reordering': len(exec_reorder_df),
        'exec_time': len(exec_time_df),
        'exec_lat': len(exec_lat_df),
        'reordering_pkl_rate(%)': 0 if len(real_lost_df) == 0 else len(lost_reorder_df)*100 / len(real_lost_df),
        'time_pkl_rate(%)': 0 if len(real_lost_df) == 0 else len(lost_time_df)*100 / len(real_lost_df),
        'real_pkl_rate(%)': 0 if len(lost_df) == 0 else len(real_lost_df)*100 / len(lost_df),
        'original_packet_loss_rate(%)': len(lost_df)*100 / len(sent_df),
        'adjusted_packet_loss_rate(%)': len(real_lost_df)*100 / len(sent_df)
    }]

    # Convert the dictionary to a dataframe
    statistics_df = pd.DataFrame.from_dict(statistics_data)
    return statistics_df

In [157]:
# exec_lat_df, exec_reorder_df, exec_time_df, real_lost_df, lost_reorder_df, lost_time_df = get_loss_data(lost_rows, processed_rcv_df)

In [158]:
all_ul_sender_files = []
all_ul_rcv_files = []
all_ul_pkl_files = []
for date in dates:
    for exp in exps:
        for device in devices:
            ul_sender_files = find_ul_sender_file(database, date, exp, device)
            ul_rcv_files = find_ul_rcv_file(database, date, exp, device)
            ul_pk_loss_files = find_ul_loss_file(database, date, exp, device)
            all_ul_sender_files.extend(ul_sender_files)
            all_ul_rcv_files.extend(ul_rcv_files)
            all_ul_pkl_files.extend(ul_pk_loss_files)

for i in range(len(all_ul_rcv_files)):
    ul_sender_df = pd.read_csv(all_ul_sender_files[i], sep=',')
    # ul_sent_df is raw file, ul_rcv_df is processed file
    ul_sent_df = ul_sender_df[(ul_sender_df['name'] == 'transport:packet_sent')]
    ul_data_df = ul_sent_df[ul_sent_df['data'].str.contains("'frame_type': 'stream'")]
    ul_rcv_df = pd.read_csv(all_ul_rcv_files[i], sep='@')
    # ul_rcver_df = pd.read_csv(all_ul_rcv_files[i], sep=',')
    # ul_rcv_df = ul_rcver_df[ul_rcver_df['name'] == 'transport:packet_received']
    # ul_rcv_df['packet_number'] = ul_rcv_df.apply(extract_packet_number, axis=1)
    ul_loss_df = pd.read_csv(all_ul_pkl_files[i])
    ul_exec_lat_df, ul_exec_reorder_df, ul_exec_time_df, ul_real_lost_df, ul_lost_reorder_df, ul_lost_time_df = get_loss_data(ul_loss_df, ul_rcv_df)
    ul_statistics = calculate_statistics(ul_lost_reorder_df, ul_lost_time_df, ul_real_lost_df, ul_exec_reorder_df, ul_exec_time_df, ul_exec_lat_df, ul_loss_df, ul_data_df, ul_sent_df)

    directory = os.path.dirname(all_ul_rcv_files[i])
    ul_loss_df['lost'] = False
    ul_loss_df['excl'] = False
    # Set 'lost' column to True for rows in ul_real_lost_df
    ul_loss_df.loc[ul_loss_df['packet_number'].isin(ul_real_lost_df['packet_number']), 'lost'] = True
    # Set 'excl' column to True for rows in ul_exec_lat_df
    ul_loss_df.loc[ul_loss_df['packet_number'].isin(ul_exec_lat_df['packet_number']), 'excl'] = True
    parts = directory.split("/")
    parts[-1] = "data"
    data_directory = "/".join(parts)
    ul_loss_df.to_csv(f"{data_directory}/ul_real_lost_pk.csv", index=False)
    # modify to the statistics directory
    parts = directory.split("/")
    parts[-1] = "statistics"
    statistics_directory = "/".join(parts)
    ul_statistics.to_csv(f"{statistics_directory}/ul_statistics.csv", index=False)

In [ ]:
all_dl_sender_files = []
all_dl_rcv_files = []
all_dl_pkl_files = []
for date in dates:
    for exp in exps:
        for device in devices:
            dl_sender_files = find_dl_sender_file(database, date, exp, device)
            dl_rcv_files = find_dl_rcv_file(database, date, exp, device)
            dl_pk_loss_files = find_dl_loss_file(database, date, exp, device)
            all_dl_sender_files.extend(dl_sender_files)
            all_dl_rcv_files.extend(dl_rcv_files)
            all_dl_pkl_files.extend(dl_pk_loss_files)
            

for i in range(len(all_dl_rcv_files)):
    dl_sender_df = pd.read_csv(all_dl_sender_files[i], sep=',')
    # ul_sent_df is raw file, ul_rcv_df is processed file
    dl_sent_df = dl_sender_df[(dl_sender_df['name'] == 'transport:packet_sent')]
    dl_data_df = dl_sent_df[dl_sent_df['data'].str.contains("'frame_type': 'stream'")]
    dl_rcv_df = pd.read_csv(all_dl_rcv_files[i], sep='@')
    dl_loss_df = pd.read_csv(all_dl_pkl_files[i])
    dl_exec_lat_df, dl_exec_reorder_df, dl_exec_time_df, dl_real_lost_df, dl_lost_reorder_df, dl_lost_time_df = get_loss_data(dl_loss_df, dl_rcv_df)
    dl_statistics = calculate_statistics(dl_lost_reorder_df, dl_lost_time_df, dl_real_lost_df, dl_exec_reorder_df, dl_exec_time_df, dl_exec_lat_df, dl_loss_df, dl_data_df, dl_sent_df)
    
    directory = os.path.dirname(all_dl_rcv_files[i])
    dl_loss_df['lost'] = False
    dl_loss_df['excl'] = False
    # Set 'lost' column to True for rows in dl_real_lost_df
    dl_loss_df.loc[dl_loss_df['packet_number'].isin(dl_real_lost_df['packet_number']), 'lost'] = True
    # Set 'excl' column to True for rows in ul_exec_lat_df
    dl_loss_df.loc[dl_loss_df['packet_number'].isin(dl_exec_lat_df['packet_number']), 'excl'] = True
    dl_loss_df.to_csv(f"{directory}/dl_real_lost_pk.csv", index=False)
    # modify to the statistics directory
    parts = directory.split("/")
    parts[-1] = "statistics"
    statistics_directory = "/".join(parts)
    dl_statistics.to_csv(f"{statistics_directory}/dl_statistics.csv", index=False)

Copy file

In [111]:
# import shutil

# def copy_file(source_file, destination_directory):
#     try:
#         shutil.copy(source_file, destination_directory)
#         print(f"File '{source_file}' copied successfully to '{destination_directory}'")
#     except FileNotFoundError:
#         print("File not found.")
#     except PermissionError:
#         print("Permission denied.")
#     except Exception as e:
#         print(f"An error occurred: {e}")

In [114]:
# for date in dates:
#     for exp in exps:
#         for device in devices:
#             exp_rounds, exp_list = exps[exp]
#             ports = device_to_port.get(device, [])
#             for exp_round in exp_list:
#                 folder_path = os.path.join(database, date, exp, device, exp_round, 'statistics')
#                 copy_file(f"{folder_path}/ul_statistics.csv", os.path.join(database, "adjusted_stats", date, exp, device, exp_round))
#                 copy_file(f"{folder_path}/dl_statistics.csv", os.path.join(database, "adjusted_stats", date, exp, device, exp_round))